In [1]:
from eve import Eve

import torch
import torch.nn as nn
from torch.autograd import Variable

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import utils
from utils import *



In [2]:
# variables
batch_size = 128
cuda = torch.cuda.is_available()

# load data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                                ])
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('data/cifar10', train=True, download=True,
                     transform=transform),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('data/cifar10', train=False, transform=transform),
    batch_size=batch_size, shuffle=True)

Files already downloaded and verified


In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, stride=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3)

        self.conv3 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3)
        self.dense1 = nn.Linear(in_features=64 * 25, out_features=512)
        self.dense1_bn = nn.BatchNorm1d(512)
        self.dense2 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(F.dropout(F.max_pool2d(self.conv2(x), 2), 0.25))
        x = F.relu(self.conv3(x))
        x = F.relu(F.dropout(F.max_pool2d(self.conv4(x), 2), 0.25))
        x = x.view(-1, 64 * 25)  # reshape
        x = F.relu(self.dense1_bn(self.dense1(x)))
        return F.log_softmax(self.dense2(x))

In [28]:
def train(epoch_index, model, optimizer):
    model.train()
    for batch_index, (data, target) in enumerate(train_loader):
        optimizer = lr_up_linearly(optimizer, epoch_index=epoch_index, batch_index=batch_index)

        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        
        def closure():
            optimizer.zero_grad()  # reset optimizer
            output = model(data)
            loss = F.nll_loss(output, target)  # negative log likelihood loss
            loss.backward()  # backprop
            return loss
        
        loss = optimizer.step(closure)
        val_accuracy = test(batch_index, model)
        val_accuracy_list.append(val_accuracy)
        if batch_index % 390 == 0:
            print('\rOne epoch train end: {} Loss: {}'.format(epoch_index, loss))
        # eval model
         
    return 'success'




In [29]:
def test(batch_index, model):
    model.eval()
    test_loss = 0
    correct = 0
    print (model.conv1.weight[0,0,0])
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = model(data)
        pred = output.data.max(1)[1]  # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()
        val_accuracy = correct / len(test_loader.dataset)

    print('\nTest set: batch_index {} Accuracy: {}/{} ({:.2%})'.format(
        batch_index, correct, len(test_loader.dataset), val_accuracy
        ))
    return val_accuracy


def plot(accuracy_list, filename):
    import matplotlib
    import numpy as np
    matplotlib.use("AGG")
    import matplotlib.pyplot as plt
    plt.plot(accuracy_list)
    plt.xlabel("Learning rate")
    plt.ylabel("Accuracy")
    plt.xticks(np.arange(0, 0.05, 0.005))
    plt.savefig(filename)
    plt.clf()
    
# linearly increase learning rate for purpose of determine the converged learning rate.
def lr_increased_linearly(epoch_index, batch_index):
    lr_max = 0.05
    batch_num = 391
    epoch_num = 8
    lr = 0.05 * (epoch_index / epoch_num) * (batch_index+1) / (batch_num)
    return lr

In [30]:
epoch_num = 8
print("sgd")
val_accuracy_list= []
model = Net()
if cuda:
    model.cuda()
optimizer = optim.SGD(model.parameters(), lr=0.05)
for epoch_index in range(1, epoch_num + 1):
    result = train(epoch_index, model, optimizer)
    print (result)

plot(val_accuracy_list, "accuracy.png")

sgd
Variable containing:
 0.1806
 0.0960
 0.0652
[torch.cuda.FloatTensor of size 3 (GPU 0)]


Test set: batch_index 0 Accuracy: 995/10000 (9.95%)
One epoch train end: 1 Loss: Variable containing:
 2.2882
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 0.1806
 0.0960
 0.0652
[torch.cuda.FloatTensor of size 3 (GPU 0)]


Test set: batch_index 1 Accuracy: 995/10000 (9.95%)
Variable containing:
 0.1806
 0.0960
 0.0652
[torch.cuda.FloatTensor of size 3 (GPU 0)]


Test set: batch_index 2 Accuracy: 995/10000 (9.95%)
Variable containing:
 0.1806
 0.0960
 0.0652
[torch.cuda.FloatTensor of size 3 (GPU 0)]


Test set: batch_index 3 Accuracy: 995/10000 (9.95%)
Variable containing:
 0.1806
 0.0960
 0.0652
[torch.cuda.FloatTensor of size 3 (GPU 0)]


Test set: batch_index 4 Accuracy: 995/10000 (9.95%)
Variable containing:
 0.1806
 0.0960
 0.0652
[torch.cuda.FloatTensor of size 3 (GPU 0)]


Test set: batch_index 5 Accuracy: 995/10000 (9.95%)
Variable containing:
 0.1806
 0.0960
 0.06


Test set: batch_index 56 Accuracy: 996/10000 (9.96%)
Variable containing:
 0.1806
 0.0960
 0.0652
[torch.cuda.FloatTensor of size 3 (GPU 0)]


Test set: batch_index 57 Accuracy: 996/10000 (9.96%)
Variable containing:
 0.1806
 0.0960
 0.0652
[torch.cuda.FloatTensor of size 3 (GPU 0)]


Test set: batch_index 58 Accuracy: 997/10000 (9.97%)
Variable containing:
 0.1806
 0.0960
 0.0652
[torch.cuda.FloatTensor of size 3 (GPU 0)]


Test set: batch_index 59 Accuracy: 994/10000 (9.94%)
Variable containing:
 0.1806
 0.0960
 0.0652
[torch.cuda.FloatTensor of size 3 (GPU 0)]


Test set: batch_index 60 Accuracy: 997/10000 (9.97%)
Variable containing:
 0.1806
 0.0960
 0.0652
[torch.cuda.FloatTensor of size 3 (GPU 0)]


Test set: batch_index 61 Accuracy: 995/10000 (9.95%)
Variable containing:
 0.1806
 0.0960
 0.0652
[torch.cuda.FloatTensor of size 3 (GPU 0)]


Test set: batch_index 62 Accuracy: 996/10000 (9.96%)
Variable containing:
 0.1806
 0.0960
 0.0652
[torch.cuda.FloatTensor of size 3 (GPU 0)]


KeyboardInterrupt: 